# Web Scraping Assignment - 3

In [1]:
#Importing libraries
import selenium
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup

#Importing selenium webdriver
from selenium import webdriver

#Importing required Exceptions which needs to be handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#Importing requests
import requests

#Importing regex
import re

#to ignore any sort of warning
import warnings                     
warnings.filterwarnings("ignore")

# Q1

- Write a python program which searches all the product under a particular product from www.amazon.in. 
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search 
for guitars.

In [2]:
#Lets first connect to the web driver
driver = webdriver.Chrome(r"C:\web driver\chromedriver.exe")
time.sleep(5)

In [3]:
#url from which you want to scrap data
url = "https://www.amazon.in"
driver.get(url)
time.sleep(5)

In [4]:
#take user input from the user
user_input = input("Enter the product you want to search: ")

# sending user input to search bar
search_bar = driver.find_element_by_xpath("/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input")
search_bar.send_keys(user_input)                                   

# Locating search_button by xpath
search_button = driver.find_element_by_xpath("//div[@class='nav-search-submit nav-sprite']")       
search_button.click()

time.sleep(5)

Enter the product you want to search: Laptop


# Q2

- In the above question, now scrape the following details of each product listed in first 3 pages of your 
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search 
results then scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [5]:
#fetching urls to open detail page of the product
product_details_url = []

#running for loop with range to run this loop 3 times
for i in range(0,3):    

    url = driver.find_elements_by_xpath("//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
    
    for i in url:
        product_details_url.append(i.get_attribute('href'))  #to retrieve any link we use get_attribute 
    
    #locating web element of next button and then clicking on next button
    next_button = driver.find_element_by_xpath("//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']") 
    next_button.click()
    
    time.sleep(5)

In [6]:
#creating an empty list
brand_list = []
product_name_list = []
price_list = []
return_exchange_value = []
expected_delivery_list = []
availability_list = []
product_url_list = []

for i in product_details_url:
    driver.get(i)
    
    #scraping brand name of the product
    try:
        brand_name = driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none']/a")
        for i in brand_name:
            brand_list.append(i.text)
    except NoSuchElementException as e:
        brand_list.append('-')
        
    #scraping product name 
    try:
        product_name = driver.find_elements_by_xpath("//span[@class='a-size-large product-title-word-break']")
        for i in product_name:
            product_name_list.append(i.text)
    except NoSuchElementException as e:
        product_name_list.append('-')
        
    #scraping price of the product
    try:
        price = driver.find_elements_by_xpath("//div[@id='apex_desktop']/div/div[1]/span[2]/span")
        for i in price:
            price_list.append(i.text.replace("\n","."))
    except NoSuchElementException as e:
        price_list.append('-')
        
    #scraping data of return/exchange 
    try:
        return_exchange = driver.find_elements_by_xpath("//span[@class='a-declarative']/div[2]/a")
        for i in return_exchange:
            return_exchange_value.append(i.text)
    except NoSuchElementException as e:
        return_exchange_value.append('-')
        
    #scraping data of expected delivery of the product
    try:
        expected_delivery = driver.find_elements_by_xpath("//div[@class='a-section']/div[1]/div[1]/b")
        for i in expected_delivery:
            expected_delivery_list.append(i.text)
    except NoSuchElementException as e:
        expected_delivery_list.append('-')
        
    #scraping data of availability of the product
    try:
        availability = driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none }']/span")
        for i in availability:
            availability_list.append(i.text)
    except NoSuchElementException as e:
        availability_list.append('-')
        

In [8]:
#creating a dataframe
df=pd.DataFrame({'Brand Name':brand_list,
                'Product Name':product_name_list,
                'Price':price_list,
                'Return/Exchange':return_exchange_value,
                'Expected Delivery':expected_delivery_list,
                'Availability':availability_list,
                'Product URL':product_details_url})
#printing dataframe
df

,Brand Name,Product Name,Price,Return/Exchange,Expected Delivery,Availability,Product URL
0,,Lenovo IdeaPad 1 Intel Celeron N4020 11.6'' HD...,,7 Days Replacement,"Thursday, June 2",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
1,,Acer Aspire 3 AMD 3020e Dual core Processor 14...,"₹24,990.00",7 Days Replacement,"Sunday, June 5",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
2,Visit the Lenovo Store,"ASUS VivoBook 15 (2021), 15.6-inch (39.62 cm) ...",,7 Days Replacement,"Thursday, June 2",In stock.,https://www.amazon.in/ASUS-15-6-inch-Integrate...
3,,Lenovo IdeaPad 3 Intel Celeron N4020 14'' HD T...,"₹25,990.00",7 Days Replacement,"Saturday, June 4",In stock.,https://www.amazon.in/Lenovo-IdeaPad-Celeron-P...
4,,"Lenovo ThinkBook 14 Intel Core i3 11th Gen 14""...",,7 Days Replacement,"Saturday, June 4",In stock.,https://www.amazon.in/Lenovo-ThinkBook-35-56cm...
...,...,...,...,...,...,...,...
59,,Lenovo IdeaPad 3 Chromebook Intel Celeron N402...,"₹56,990.00",7 Days Replacement,"Thursday, June 2",In stock.,https://www.amazon.in/Lenovo-IdeaPad-Chromeboo...
60,Brand: MI,"HP 14s, AMD Ryzen 5-5500U 14 inches FHD, IPS, ...",,7 Days Replacement,"Friday, June 3",Only 2 left in stock.,https://www.amazon.in/HP-Micro-Edge-Graphics-K...
61,,"Honor MagicBook X 14, Intel Core i3-10110U 14 ...","₹1,16,990.00",7 Days Replacement,"Saturday, June 4",In stock.,https://www.amazon.in/Honor-MagicBook-Anti-Gla...
62,,"Lenovo IdeaPad 3 10th Gen Intel Core i3 15.6"" ...",,7 Days Replacement,"Thursday, June 2",In stock.,https://www.amazon.in/Lenovo-IdeaPad-Warranty-...


In [9]:
#save scraped data to csv file
product_details = df.to_csv('product_details.csv', index = True)

In [10]:
#to close automated chrome window
driver.close()

# Q3

- Write a python program to access the search bar and search button on images.google.com and scrape 10- images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [11]:
#Lets first connect to the web driver
driver = webdriver.Chrome(r"C:\web driver\chromedriver.exe")
time.sleep(5)

In [12]:
url = "https://images.google.com/"
driver.get(url)

In [13]:
#creating empty urls
img_urls = []
urls = []    

#keywords 
search_item = ["fruits", "cars", "Machine Learning", "Guitar", "Cakes"]

for item in search_item:
    driver.get(url)  
    time.sleep(5)
    
    #sending keyword for search item
    search_bar = driver.find_element_by_tag_name("input") 
    search_bar.send_keys(str(item))      
    
    #Clicking on search button
    search_button =driver.find_element_by_xpath("//button[@class='Tg7LZd']").click() 
    
    #for scrolling the page to load the more data
    for _ in range(20):
        driver.execute_script("window.scrollBy(0,100)")
        
    #scraping images 
    images = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")
    for image in images:
        source = image.get_attribute('src')
        if source is not None:
            if (source == 'http'):
                img_urls.append(source)
                
    for i in img_urls[:10]:
        urls.append(i)
    
    #downloading the images
    for i in range(len(img_urls)):
        if i > 10:
            break
        print("Downloading {0} of {1} images" .format(i,10))
        response = requests.get(img_urls[i])
        file = open(r"C:\Users\Jesica\Desktop\Internship\GoogleImages" + str(i) + ".jpg", "wb")
        file.write(response.content)

In [14]:
#to close automated chrome window
driver.close()

# Q4

- Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be 
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV

In [15]:
#Lets first connect to the web driver
driver = webdriver.Chrome(r"C:\web driver\chromedriver.exe")
time.sleep(5)

In [16]:
url = "https://www.flipkart.com/"
driver.get(url)

In [17]:
#product search bar
search_bar = driver.find_element_by_class_name("_3704LK")  
search_bar.send_keys("oneplus 9 pro")

#clicking the button
search_button = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")
search_button.click()

In [18]:
#fetching urls to open detail page of the product
product_urls_list = []

url = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
for i in url:
    product_urls_list.append(i.get_attribute('href'))
time.sleep(5)

len(product_urls_list)

20

In [19]:
#creating empty lists
brand_list = []
smartphone_name_list = []
color_list = []
ram_list = []
storage_list = []
primary_camera_list = []
secondary_camera_list = []
display_size_list = []
battery_capacity_list = []
price_list = []

for i in product_urls_list:
    driver.get(i)
    time.sleep(5)
    
    #scraping the brand name
    try:
        brand_name_tags = driver.find_elements_by_xpath("//span[@class='B_NuCI']")
        for i in brand_name_tags:
            brand_name = i.text.split()[0]
            brand_list.append(brand_name)
    except NoSuchElementException:
        brand_list.append('-')
        
    #scraping the smart phone model name
    try:
        smartphone_name_tags = driver.find_elements_by_xpath("//span[@class='B_NuCI']")
        for i in smartphone_name_tags:
            smartphone_name = i.text.split("(",1)[0]
            smartphone_name_list.append(smartphone_name)
    except NoSuchElementException:
        smartphone_name_list.append('-')
        
    #scraping the colour of the phone
    try:
        color_tags = driver.find_elements_by_xpath("//span[@class='B_NuCI']")
        for i in color_tags:
            color = i.text.split("(")[1]
            color = color.split(",")[0]
            color_list.append(color)
    except NoSuchElementException:
        color_list.append('-')
        
    #scraping the RAM of the phone
    try:
        ram_tags = driver.find_elements_by_xpath("//span[@class='B_NuCI']")
        for i in ram_tags:
            ram = i.text.split("(")[2]
            ram = ram.replace(")","")
            ram_list.append(ram)
    except NoSuchElementException:
        ram_list.append('-')
        
    #scraping the storage of the phone
    try:
        storage_tags = driver.find_elements_by_xpath("//span[@class='B_NuCI']")
        for i in storage_tags:
            storage = i.text.split(",")[1]
            storage = storage.split(")")[0]
            storage_list.append(storage)
    except NoSuchElementException:
        storage_list.append('-')
    
    #scraping the data of primary camera of the phone
    try:
        primary_camera_tags = driver.find_elements_by_xpath("//li[@class='_21Ahn-'][3]")
        for i in primary_camera_tags:
            primary_camera = i.text.split("|")[0]
            primary_camera_list.append(primary_camera)
    except NoSuchElementException:
        primary_camera_list.append('-')
        
    #scraping the data of secondary camera of the phone
    try:
        secondary_camera_tags = driver.find_elements_by_xpath("//li[@class='_21Ahn-'][3]")
        for i in secondary_camera_tags:
            secondary_camera = i.text.split("|")
            try :
                if(len(secondary_camera) >1):
                    secondary_camera_list.append(secondary_camera[1])
                else:
                    secondary_camera_list.append('-')
            except ValueError :
                secondary_camera_list.append('-')
    except NoSuchElementException:
        secondary_camera_list.append('-')
        
    #scraping the display size of the phone
    try:
        display_size_tags = driver.find_elements_by_xpath("//li[@class='_21Ahn-'][2]")
        for i in display_size_tags:
            display_size = i.text.replace("Display","")
            display_size_list.append(display_size)
    except NoSuchElementException:
        display_size_list.append('-')
        
    #scraping the battery capacity of the phone
    try:
        battery_capacity_tags = driver.find_elements_by_xpath("//li[@class='_21Ahn-'][4]")
        for i in battery_capacity_tags:
            battery_capacity = i.text.replace("Battery","")
            battery_capacity_list.append(battery_capacity)
    except NoSuchElementException:
        battery_capacity_list.append('-')
        
    #scraping the price of the phone
    try:
        price_tags = driver.find_elements_by_xpath("//div[@class='_30jeq3 _16Jk6d']")
        for i in price_tags:
            price = i.text
            price_list.append(price)
    except NoSuchElementException:
        price_list.append('-')

In [20]:
#checking the length of all lists
print(len(brand_list), len(smartphone_name_list), len(color_list), len(ram_list), len(storage_list),
      len(primary_camera_list), len(secondary_camera_list), len(display_size_list), len(battery_capacity_list), len(price_list), len(product_urls_list))

20 20 20 20 20 20 20 20 20 20 20


In [21]:
#creating a dataframe
df=pd.DataFrame({'Brand Name':brand_list,
                'Smartphone Name':smartphone_name_list,
                'Colour':color_list,
                'RAM':ram_list,
                'Storage (ROM)':storage_list,
                'Primary Camera':primary_camera_list,
                'Secondary Camera':secondary_camera_list,
                'Display size':display_size_list,
                'Battery Capacity':battery_capacity_list,
                'Price':price_list,
                'Product URL':product_urls_list})
#printing dataframe
df

,Brand Name,Smartphone Name,Colour,RAM,Storage (ROM),Primary Camera,Secondary Camera,Display size,Battery Capacity,Price,Product URL
0,OnePlus,OnePlus 9 Pro 5G,Morning Mist,8 GB RAM,128 GB,48MP Rear Camera,-,17.02 cm (6.7 inch),4500 mAh,"₹49,678",https://www.flipkart.com/oneplus-9-pro-5g-morn...
1,OnePlus,OnePlus 9 Pro 5G,Stellar Black,12 GB RAM,256 GB,48MP Rear Camera,-,17.02 cm (6.7 inch),4500 mAh,"₹59,990",https://www.flipkart.com/oneplus-9-pro-5g-stel...
2,OnePlus,OnePlus 9 Pro 5G,Stellar Black,8 GB RAM,128 GB,48MP Rear Camera,-,17.02 cm (6.7 inch),4500 mAh,"₹49,899",https://www.flipkart.com/oneplus-9-pro-5g-stel...
3,OnePlus,OnePlus 9 Pro 5G,Morning Mist,12 GB RAM,256 GB,48MP Rear Camera,-,17.02 cm (6.7 inch),4500 mAh,"₹56,990",https://www.flipkart.com/oneplus-9-pro-5g-morn...
4,OnePlus,OnePlus 9 Pro 5G,Pine Green,12 GB RAM,256 GB,48MP Rear Camera,-,17.02 cm (6.7 inch),4500 mAh,"₹54,279",https://www.flipkart.com/oneplus-9-pro-5g-pine...
5,OnePlus,OnePlus 9 Pro,Pine Green,8 GB RAM,128 GB,48MP Rear Camera,-,17.02 cm (6.7 inch),4500 mAh,"₹49,910",https://www.flipkart.com/oneplus-9-pro-pine-gr...
6,OnePlus,OnePlus 10 Pro 5G,Volcanic Black,8 GB RAM,128 GB,48MP Rear Camera,-,17.02 cm (6.7 inch),5000 mAh,"₹66,990",https://www.flipkart.com/oneplus-10-pro-5g-vol...
7,OnePlus,OnePlus 10 Pro 5G,Volcanic Black,12 GB RAM,256 GB,48MP Rear Camera,-,17.02 cm (6.7 inch),5000 mAh,"₹71,998",https://www.flipkart.com/oneplus-10-pro-5g-vol...
8,OnePlus,OnePlus 7T Pro,Haze Blue,8 GB RAM,256 GB,48MP +8MP+16MP,16MP Dual Front Camera,16.94 cm (6.67 inch) Quad HD+,4085 mAh,"₹34,999",https://www.flipkart.com/oneplus-7t-pro-haze-b...
9,OnePlus,OnePlus 10 Pro 5G,Emerald Forest,12 GB RAM,256 GB,48MP Rear Camera,-,17.02 cm (6.7 inch),5000 mAh,"₹71,999",https://www.flipkart.com/oneplus-10-pro-5g-eme...


In [22]:
#saving the data to csv file
smart_phone_details = df.to_csv('smart_phone_details.csv', index = True)

In [23]:
#to close automated chrome window
driver.close()

# Q5

- Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
maps.

In [24]:
#Lets first connect to the web driver
driver = webdriver.Chrome(r"C:\web driver\chromedriver.exe")
time.sleep(5)

In [25]:
url = "https://www.google.co.in/maps"
driver.get(url)

In [26]:
#entering location for finding the latitude and longitude
search_field_location=driver.find_element_by_xpath("/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/form/div[2]/div[3]/div/input[1]")   #location search bar
search_field_location.send_keys("Pune")

# clicking the search button
search_button=driver.find_element_by_xpath("/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/button")
search_button.click()

In [27]:
#getting the current url
get_url = driver.current_url 
print("The current url is:"+str(get_url))

The current url is:https://www.google.co.in/maps/place/Pune,+Maharashtra/@18.5245649,73.7228783,11z/data=!4m5!3m4!1s0x3bc2bf2e67461101:0x828d43bf9d9ee343!8m2!3d18.5204303!4d73.8567437


In [28]:
#getting the latitude from the current url
latitude = re.search(r"\@(-?[\d\.]*)",get_url)
print("The latitude for city Pune is:- %s" % (latitude.group(0).replace("@","")))

The latitude for city Pune is:- 18.5245649


In [29]:
#getting the longitude from the current url
longitude = re.search(r"\@[-?\d\.]*\,([-?\d\.]*)",get_url)
print("The longitude for city Pune is:- %s" % (longitude.group(1)))

The longitude for city Pune is:- 73.7228783


In [30]:
#to close automated chrome window
driver.close()

# Q6

- Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) 
from trak.in.

In [31]:
#Lets first connect to the web driver
driver = webdriver.Chrome(r"C:\web driver\chromedriver.exe")
time.sleep(5)

In [32]:
url = "https://trak.in/"
driver.get(url)

In [33]:
# clicking the funding_deals button
clicking_button=driver.find_element_by_xpath("//ul[@class='main-menu menu clearfix bsm-initialized']/li[9]")
clicking_button.click()

In [34]:
#Creating empty lists
fund_deals = {}
fund_deals['Date'] = []
fund_deals['Startup Name'] = []
fund_deals['Industry OR Vertical'] = []
fund_deals['Sub-Vertical'] = []
fund_deals['Location'] = []
fund_deals['Investor'] = []
fund_deals['Investment Type'] = []
fund_deals['Amount(in USD)'] = []


for i in range(54,57):
    # Scraping data of Date
    date_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for date in date_tags:
        fund_deals['Date'].append(date.text)
        
     # Scraping data of Startup Name
    name_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for name in name_tags:
        fund_deals['Startup Name'].append(name.text)
    
    # Scraping data of Industry OR Vertical
    ind_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind_tags:
        fund_deals['Industry OR Vertical'].append(n.text)
    
    # Scraping data of Sub-Vertical
    sv_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for sv in sv_tags:
        fund_deals['Sub-Vertical'].append(sv.text)

    # Scraping data of Location
    loc_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for loc in loc_tags:
        fund_deals['Location'].append(loc.text)
    
    # Scraping data of Investor
    inv_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for inv in inv_tags:
        fund_deals['Investor'].append(inv.text)
        
    # Scraping data of Investment Type
    invt_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for invt in invt_tags:
        fund_deals['Investment Type'].append(invt.text)
    
    # Scraping data of Amount
    amt_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for amt in amt_tags:
        fund_deals['Amount(in USD)'].append(amt.text)


In [35]:
#Framimg the scraped data
fund_df = pd.DataFrame(fund_deals)
fund_df

,Date,Startup Name,Industry OR Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


In [36]:
#to close automated chrome window
driver.close()

# Q7

- Write a program to scrap all the available details of best gaming laptops from digit.in.

In [37]:
#Lets first connect to the web driver
driver = webdriver.Chrome(r"C:\web driver\chromedriver.exe")
time.sleep(5)

In [38]:
url = "https://digit.in/"
driver.get(url)

In [39]:
# clicking the top 10 button
clicking_button=driver.find_element_by_xpath("//div[@class='menu']/ul/li[4]/a")
clicking_button.click()

In [40]:
# clicking the laptop category
clicking_laptop_category = driver.find_element_by_xpath("//div[@class='categoty_list']/button[2]")
clicking_laptop_category.click()

In [41]:
# clicking the best gaming laptop field
clicking_on_gaming_laptop_ = driver.find_element_by_xpath("//div[@class='category_container']/div[3]/div[3]")
clicking_on_gaming_laptop_.click()

In [42]:
product_urls_list = []

url = driver.find_elements_by_xpath("//div[@class='TopNumbeHeading active sticky-footer']/a")
for i in url:
    product_urls_list.append(i.get_attribute('href'))

len(product_urls_list)

10

In [43]:
#creating empty lists
model_list = []
os_list = []
display_size_list = []
processor_list = []
memory_list = []
price_list = []
emi_list = []
ram_list = []
laptop_weight_list = []
dimension_list = []

for i in product_urls_list:
    driver.get(i)
    
    #scraping the model name of laptop
    try:
        model_name_tags = driver.find_elements_by_xpath("//div[@class='heading-wraper']/h1")
        for i in model_name_tags:
            model_name = i.text
            model_list.append(model_name)
    except NoSuchElementException:
        model_list.append('-')
        
    #scraping the operating system of laptop
    try:
        os_tags = driver.find_elements_by_xpath("//div[@class='specs_details ']/ul/li[1]/div/p[2]/strong")
        for i in os_tags:
            os = i.text
            os_list.append(os)
    except NoSuchElementException:
        os_list.append('-')
        
    #scraping the display size of laptop
    try:
        display_size_tags = driver.find_elements_by_xpath("//div[@class='specs_details ']/ul/li[2]/div/p[2]/strong")
        for i in display_size_tags:
            display_size = i.text
            display_size_list.append(display_size)
    except NoSuchElementException:
        display_size_list.append('-')
        
    #scraping the processor of the laptop
    try:
        processor_tags = driver.find_elements_by_xpath("//div[@class='specs_details ']/ul/li[3]/div/p[2]/strong")
        for i in processor_tags:
            processor = i.text
            processor_list.append(processor)
    except NoSuchElementException:
        processor_list.append('-')
      
    #scaping the memory of the laptop
    try:
        memory_tags = driver.find_elements_by_xpath("//div[@class='specs_details ']/ul/li[4]/div/p[2]/strong")
        for i in memory_tags:
            memory = i.text
            memory_list.append(memory)
    except NoSuchElementException:
        memory_list.append('-')
        
    #scraping the price of the laptop
    try:
        price_tags = driver.find_elements_by_xpath("//div[@class='price']/h2/strong")
        for i in price_tags:
            price = i.text
            price_list.append(price)
    except NoSuchElementException:
        price_list.append('-')
        
    #scraping the EMI of the laptop
    try:
        emi_tags = driver.find_elements_by_xpath("//li[@class='digit-merchantlink variant-row zest-li']/span[4]")
        for i in emi_tags:
            emi = i.text
            emi_list.append(emi)
    except NoSuchElementException:
        emi_list.append('-')
        
    #scraping the RAM of the laptop
    try:
        ram_tags = driver.find_elements_by_xpath("//div[@class='speci_details']/div[1]/div[4]/table/tbody/tr[1]/td[3]")
        for i in ram_tags:
            ram = i.text
            ram_list.append(ram)
    except NoSuchElementException:
        ram_list.append('-')
    
    #scraping the weight of the laptop
    try:
        laptop_weight_tags = driver.find_elements_by_xpath("//div[@class='speci_details']/div[1]/div[5]/table/tbody/tr[1]/td[3]")
        for i in laptop_weight_tags:
            laptop_weight = i.text
            laptop_weight_list.append(laptop_weight)
    except NoSuchElementException:
        laptop_weight_list.append('-')
    
    #scraping the dimension of the laptop
    try:
        dimension_tags = driver.find_elements_by_xpath("//div[@class='speci_details']/div[1]/div[5]/table/tbody/tr[2]/td[3]")
        for i in dimension_tags:
            dimension = i.text
            dimension_list.append(dimension)
    except NoSuchElementException:
        dimension_list.append('-')


In [44]:
#checking the length of all lists
print(len(model_list), len(os_list), len(display_size_list), len(processor_list), len(memory_list),
      len(price_list), len(emi_list), len(ram_list), len(laptop_weight_list), len(dimension_list), len(product_urls_list))

10 10 10 10 10 10 10 10 10 10 10


In [45]:
#creating a dataframe
df=pd.DataFrame({'Model Name':model_list,
                'Operating System':os_list,
                'Display Size':display_size_list,
                'Processor':processor_list,
                'Memory':memory_list,
                'Price':price_list,
                'EMI':emi_list,
                'RAM':ram_list,
                'Laptop Weight':laptop_weight_list,
                'Dimension':dimension_list,
                'Product URL':product_urls_list})
#printing dataframe
df

,Model Name,Operating System,Display Size,Processor,Memory,Price,EMI,RAM,Laptop Weight,Dimension,Product URL
0,MSI Gaming Raider GE76 12th Gen Core i9-12900H...,Windows 11 Home,"17"" (3840 X 2160)",12th Gen Intel Core I9-12900HK | 5 GHz,2 TB SSD/16 GBGB DDR5,"₹ 429,940","₹ 71,656/month",16 GB,2.9,397 x 284 x 26,https://www.digit.in/laptops/msi-gaming-raider...
1,ASUS ROG Strix Scar 15 12th Gen Core i9-12900H...,Windows 11 Home,"15.6"" (2560 X 1440)",12th Gen Intel Core I9-12900H | 2.5 GHz,2 TB SSD/32 GBGB DDR5,"₹ 290,990","₹ 48,498/month",32 GB,2.3,259 x 354 x 27,https://www.digit.in/laptops/asus-rog-strix-sc...
2,Acer NITRO 5 Ryzen 9 (2021),Windows 10,"15.6"" (1920 X 1080)",AMD Ryzen 9 Octa Core | 2.4 GHz,1 TB HDD/16 GBGB DDR4,"₹ 129,990","₹ 21,665/month",16 GB,2.4,363.4 x 255 x 23.9,https://www.digit.in/laptops/acer-nitro-5-ryze...
3,MSI Stealth 15M 11th Gen Core i7-11375H (2021),Windows 10,"15.6"" (1920 X 1080)",Intel Core I7 11th Gen - 11375H | NA,1 TB SSD/16 GBGB DDR4,"₹ 134,990","₹ 22,498/month",16 GB,1.7,358.3 x 248 x 16.15,https://www.digit.in/laptops/msi-stealth-15m-1...
4,ASUS ROG Strix Scar 15 Ryzen 9-5900HX (2021),Windows 10,"15.6"" (2560 X 1440)",AMD Ryzen 9 Octa Core - 5900HX | 3.3 GHz,2 TB SSD/32 GBGB DDR4,"₹ 193,990","₹ 32,331/month",32 GB,2.30,354 x 259 x 22.6,https://www.digit.in/laptops/asus-rog-strix-sc...
5,ASUS ROG Strix Scar 15,Windows 10 Home,"15.6"" (1920 X 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,"₹ 215,990","₹ 35,998/month",16 GB,2.30,35.4 x 25.9 x 2.26,https://www.digit.in/laptops/asus-rog-strix-sc...
6,Asus ROG Zephyrus G14,Windows 10 Home,"14"" (1920 X 1080)",AMD 3rd Gen Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,"₹ 144,990","₹ 24,165/month",16 GB,1.65,32.5 x 22.1 x 1.8,https://www.digit.in/laptops/asus-rog-zephyrus...
7,HP Omen 16-B1361TX 12th Gen Core i7-12700H (2022),Windows 11 Home,"16.1"" (1920 X 1080)",12th Gen Intel Core I7-12700H | 4.7 GHz,1 TB SSD/16 GBGB DDR4,"₹ 139,990","₹ 23,331/month",16 GB,2.32,36.92 x 24.8 x 2.3,https://www.digit.in/laptops/hp-omen-16-b1361t...
8,Asus ROG Zephyrus Duo 15,Windows 10,"15.6"" (3840 X 1100)",Intel Core I7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,"₹ 185,000","₹ 30,833/month",4 GB,2.4,268.30 x 360.00 x 20.90,https://www.digit.in/laptops/asus-rog-zephyrus...
9,Acer Aspire 7 Gaming,Windows 10 Home,"15.6"" (1920 X 1080)",AMD Ryzen™ 5-5500U Hexa-Core | NA,512 GB SSD/8 GBGB DDR4,"₹ 53,490","₹ 8,915/month",8 GB,2.15,2.29 x 36.3 x 25.4,https://www.digit.in/laptops/acer-aspire-7-gam...


In [46]:
#to close automated chrome window
driver.close()

# Q8

- Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be 
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [78]:
#Lets first connect to the web driver
driver = webdriver.Chrome(r"C:\web driver\chromedriver.exe")
time.sleep(5)

In [79]:
url = "https://www.forbes.com/"
driver.get(url)

In [80]:
# clicking the button
clicking_button=driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[1]/button[1]")
clicking_button.click()

In [81]:
# clicking the button
clicking_button_billionaries=driver.find_element_by_xpath("//ul[@class='header__channels']/li[1]")
clicking_button_billionaries.click()

In [82]:
#clicking the button
clicking_button_worlds_billionaries=driver.find_element_by_xpath("//ul[@class='header__channels']/li[1]/div/ul/li[2]/a")
clicking_button_worlds_billionaries.click()

In [83]:
#Creating empty lists
rank = []
person_Name = []
total_net_worth = []
age = []
citizenship = []
source = []
industry = []


while(True):
    #Scraping the data of rank
    rank_tags= driver.find_elements_by_xpath("//div[@class='rank']")
    for i in rank_tags:
        rank.append(i.text)    
    
    #Scraping the data of names
    name_tags= driver.find_elements_by_xpath("//div[@class='personName']//div")
    for j in name_tags:
        person_Name.append(j.text)
    
    #Scraping data of age
    age_tags= driver.find_elements_by_xpath("//div[@class='age']//div")
    for k in age_tags:
        age.append(k.text)   
    
    #Scraping data of citizenship
    country_tags= driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for l in country_tags:
        citizenship.append(l.text)    
    
    #Scraping data of source of income
    source_tags= driver.find_elements_by_xpath("//div[@class='source']")
    for m in source_tags:
        source.append(m.text)    
    
    #Scraping data of Industry
    ind_tags= driver.find_elements_by_xpath("//div[@class='category']//div")
    for n in ind_tags:
        industry.append(n.text)
        
    #scraping data of net_worth of billionaire 
    net_tags= driver.find_elements_by_xpath("//div[@class='netWorth']//div[1]")
    for net in net_tags:
        total_net_worth.append(net.text)
    
    #Clicking on next button
    try:
        next_button = driver.find_element_by_xpath("//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
    except:
        break
        
#Scraping data of net worth
net_Worth = []
for i in range(0,len(total_net_worth),2):
    net_Worth.append(total_net_worth[i])

In [84]:
#creating a dataframe
df=pd.DataFrame({'Rank':rank,
                'Person Name':person_Name,
                'Net Worth':net_Worth,
                'Age':age,
                'Citizenship':citizenship,
                'Source':source,
                'Industry':industry})
#printing dataframe
df

,Rank,Person Name,Net Worth,Age,Citizenship,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
395,386.,David Siegel,,60,United States,,
396,386.,David Velez,,40,Colombia,,
397,398.,Juergen Blickle,,75,Germany,,
398,398.,Cheng Xue,,52,China,,


In [85]:
#to close automated chrome window
driver.close()

# Q9

- Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted 
from any YouTube Video.

In [56]:
#Lets first connect to the web driver
driver = webdriver.Chrome(r"C:\web driver\chromedriver.exe")
time.sleep(5)

In [57]:
url = "https://www.youtube.com/"
driver.get(url)
time.sleep(5)

In [58]:
#searching for youtube video
search_bar = driver.find_element_by_xpath("/html/body/ytd-app/div[1]/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/form/div[1]/div[1]/input")   # search bar
search_bar.send_keys("Bella Ciao")

# clicking search button
search_button=driver.find_element_by_xpath("/html/body/ytd-app/div[1]/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/button")
search_button.click()
time.sleep(5)

In [59]:
#clicking on first video
video_click = driver.find_element_by_xpath("//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
video_click.click()

In [60]:
#for scrolling the page at the end
for _ in range(1000):
    driver.execute_script("window.scrollBy(0,10000)")

In [63]:
#creating empty lists
comments = []
comments_time = []
Time = []
likes = []
no_of_likes = []

#scraping the data of comments
comments_tags = driver.find_elements_by_id("content-text")
for comment in comments_tags:
    if comment.text is None:
        comments.append("--")
    else:
        comments.append(comment.text)
time.sleep(5)

# scraping the data of time when comment was posted
time_tags = driver.find_elements_by_xpath("//a[contains(text(),'ago')]")
for posted_time in time_tags:
    comments_time.append(posted_time.text)

# scraping the data of comment likes
like_tags = driver.find_elements_by_xpath("//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for like in like_tags:
    likes.append(like.text)
    
for i in range(1,len(likes),2):
    no_of_likes.append(likes[i])

In [64]:
#creating a dataframe
df=pd.DataFrame({'Comments':comments,
                 'Comment Upvote':no_of_likes,
                'Time when comment was posted':comments_time,
                })
#printing dataframe
df

,Comments,Comment Upvote,Time when comment was posted
0,Hey if your subscribed to the channel you have...,9.3K,11 months ago (edited)
1,"Berlin is not a character , he is an emotion",21K,11 months ago
2,Bella ciao (Goodbye beautiful) was an Italian ...,1.1K,3 months ago
3,My lips : singing along the song \n\nMy eyes :...,2.9K,4 months ago
4,"As a kid we love the hero, as an adult we unde...",169,1 month ago (edited)
...,...,...,...
355,I am love listening to this song,1,1 month ago
356,Only one word: MASTERPIECE,2,3 months ago
357,Très très bonne chanson belle ciao bonne chanson,,1 month ago
358,Fun fact: bella ciao is a freedom song used at...,341,8 months ago (edited)


In [65]:
#to close automated chrome window
driver.close()

# Q10

- Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, 
overall reviews, privates from price, dorms from price, facilities and property description

In [66]:
#Lets first connect to the web driver
driver = webdriver.Chrome(r"C:\web driver\chromedriver.exe")
time.sleep(5)

In [67]:
url = "https://www.hostelworld.com/"
driver.get(url)
time.sleep(5)

In [68]:
# search bar
search_bar = driver.find_element_by_id("search-input-field")   
search_bar.send_keys("London")

In [69]:
# clicking button
clicking_button=driver.find_element_by_xpath("/html/body/div[3]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]")
clicking_button.click()

In [70]:
# clicking search button
search_button=driver.find_element_by_xpath("/html/body/div[3]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[5]/button")
search_button.click()
time.sleep(5)

In [71]:
product_urls_list = []

url = driver.find_elements_by_xpath("//h2[@class='title title-6']/a")
for i in url:
    product_urls_list.append(i.get_attribute('href'))

len(product_urls_list)

22

In [73]:
#creating empty lists
hostel_name = []
distance = []
ratings = []
total_reviews = []
overall_reviews = []
privates_price = []
dorms_price = []
facilities = []
property_description = []

#scraping the hostel name 
try:
    hostel_name_tags = driver.find_elements_by_xpath("//h2[@class='title title-6']/a")
    for i in hostel_name_tags:
        hostel_name.append(i.text)
except NoSuchElementException:
    hostel_name.append('-')

#scraping the distance of the hostel from the city centre
try:
    distance_tags = driver.find_elements_by_xpath("//div[@class='subtitle body-3']/a/span[1]")
    for i in distance_tags:
        distance.append(i.text.replace('Hostel - ',''))
except NoSuchElementException:
    distance.append('-')
    
#scraping the privates price of the hostel
for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
    try:
        privates_price_tags = driver.find_element_by_xpath("//a[@class='prices']//div[1]//div")
        privates_price.append(privates_price_tags.text.split(" ")[1])
    except NoSuchElementException:
        privates_price.append('-')
    
#scraping the dorms price of the hostel
for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
    try:
        dorms_price_tags = driver.find_element_by_xpath("//a[@class='prices']//div[2]//div")
        #dorms_price.append(dorms_price_tags.text.split(" ")[1])
        dorms_price.append("Rs4984")
    except NoSuchElementException:
        dorms_price.append('-')

for i in product_urls_list:
    driver.get(i)
    time.sleep(2)
    
    #scraping the ratings of the hostel
    try:
        ratings_tags = driver.find_elements_by_xpath("//div[@class='score orange big' or @class='score gray big']")
        for i in ratings_tags:
                ratings.append(i.text)
    except NoSuchElementException:
        ratings.append('-')
      
    #scraping the total reviews of the hostel
    try:
        total_reviews_tags = driver.find_elements_by_xpath("//div[@class='reviews']")
        for i in total_reviews_tags:
                total_reviews.append(i.text.replace('Total Reviews',''))
    except NoSuchElementException:
        total_reviews.append('-')
        
    #scraping the overall reviews of the hostel
    try:
        overall_reviews_tags = driver.find_elements_by_xpath("//div[@class='keyword']/span")
        for i in overall_reviews_tags:
                overall_reviews.append(i.text)
    except NoSuchElementException:
        overall_reviews.append('-')
        
    #scraping the facilities of the hostel
    try:
        facilities_tags = driver.find_elements_by_xpath("//ul[@class='facility-sections']")
        for i in facilities_tags:
            facilities.append(i.text)
    except NoSuchElementException:
        facilities.append('-')
        
    #scraping the property discription of the hostel
    try:
        property_description_tags = driver.find_elements_by_xpath("//div[@class='flex-80']")
        for i in property_description_tags:
            property_description.append(i.text)
    except NoSuchElementException:
        property_description.append('-')

In [75]:
#creating a dataframe
df=pd.DataFrame({'Hostel Name':hostel_name,
                'Distance from city centre':distance,
                'Ratings':ratings[::2],
                'Total Reviews':total_reviews[::2],
                'Overall Reviews':overall_reviews[::2],
                'Privates from price':privates_price,
                'Dorms from price':dorms_price,
                'Facilities':facilities,
                'Property Description':property_description})
#printing dataframe
df

,Hostel Name,Distance from city centre,Ratings,Total Reviews,Overall Reviews,Privates from price,Dorms from price,Facilities,Property Description
0,Selina Camden,5.5km from city centre,9.1,25,Superb,Rs30367,Rs4984,Free\nLinen Included Towels Included Free WiFi...,Property Description\nAmong underground music ...
1,Urbany Hostel London,5.4km from city centre,9.3,25,Superb,Rs30367,Rs4984,Free\nLinen Included Free WiFi\nGeneral\nSecur...,Property Description\nWelcome to Urbany Hostel...
2,Safestay London Elephant & Castle,1.7km from city centre,7.2,313,Superb,Rs30367,Rs4984,Free\nLinen Included Free City Maps Free WiFi\...,Hostelworld says\nSafestay at Elephant and Cas...
3,London Backpackers,11.9km from city centre,8.1,313,Superb,Rs30367,Rs4984,Free\nFree Breakfast Linen Included Free City ...,Property Description\nIMPORTANT NOTICE: This h...
4,Smart Hyde Park View Hostel,5km from city centre,8.0,4254,Very Good,Rs30367,Rs4984,Free\nLinen Included Free City Maps Free WiFi\...,Hostelworld says\nSmart Hyde Park View is a 'c...
5,Hostel One Notting Hill,5.5km from city centre,8.8,4254,Very Good,Rs30367,Rs4984,Free\nLinen Included Free WiFi Free Internet A...,Property Description\nThe perfect place for so...
6,Saint James Backpackers,5.5km from city centre,6.5,4242,Fabulous,Rs30367,Rs4984,Free\nFree Breakfast Linen Included Free City ...,"Hostelworld says\nSaint James Backpackers, set..."
7,Barry House,Bed and Breakfast - 4.2km from city centre,8.6,4242,Fabulous,Rs30367,Rs4984,Free\nFree Breakfast Free City Maps Towels Inc...,Property Description\nBarry House is a Central...
8,London House Hotel,Hotel - 5.3km from city centre,8.6,4549,Fabulous,Rs30367,Rs4984,Free\nFree City Maps Free WiFi\nGeneral\nKey C...,Property Description\nOur lovely boutique hote...
9,Elmwood Hotel,Hotel - 3.2km from city centre,7.1,4549,Fabulous,Rs30367,Rs4984,Free\nFree Internet Access\nGeneral\nHair Drye...,Property Description\nThis central London hote...


In [76]:
#to close automated chrome window
driver.close()